## Substitution boxes analysis

author: Szymon Krysztopolski 144619

### Block ciphers
In cryptography, one of the most popular types of cipers are block ciphers. These algorythms operate on fixed-length groups of bytes, called blocks. This type of ciphers is based on symmetric keys. The most popular types of block ciphers are DES (Data Endryption Standard) and AES (Advanced Encryption Standard).

### S-box
S-box is basis of block ciphers, it is kind of black-box. S-box is the only nonlinear element of a cipher. In block ciphers, they are typically used to obscure the relationship between the key and the ciphertext. Relation between input and output data is difficult to predict, since a small change in a single input bit has impact to about half of output bits.

![](./img/sbox_table.png)

![](./img/sbox_rule.jpg)

In [1]:
import numpy as np

FileName = "sbox_08x08.sbx"
SBoxLen = 8
SBoxFunLen = 256

We use `sbox_08x08.sbx` s-box file to analysis.

![](./img/sbox8.png)

In [2]:
def getbit(x: list, n: int) -> bool:
    return x & (1 << n) and 1 or 0

def bitfield(n: int, bits: int) -> list:
    return [int(digit) for digit in (bin(n)[2:]).zfill(bits)]

def generateFunctions(SBoxLen: int, SBoxFunLen: int) -> np.array:
    return np.array([bitfield(i, SBoxLen) for i in range(SBoxFunLen)])

In [3]:
def createFunctions() -> np.array:
    with open(FileName, mode='rb') as file: 
        fileContent = file.read()
    
    resultTab = np.empty((0, SBoxFunLen), dtype=np.int32)
    for i in range(SBoxLen):
        nextFun = [int(getbit(fileContent[j], i)) for j in range(0, len(fileContent), 2)]
        resultTab = np.append(resultTab, [np.array(nextFun)], axis=0)
    
    return resultTab

In [4]:
funArray = createFunctions()
print(np.sum(funArray, axis=1))

[128 128 128 128 128 128 128 128]


Number of ones of every single function od equal to 128 (exactly half). This means that S-box is `balanced`.

In [5]:
def xorTree(bestScore: int, currentFun: np.array, testArray: np.array) -> float:
    if testArray.shape[0] <= 1:
        return np.inf

    for i in range(len(testArray)):
        newFunArray = np.bitwise_xor(currentFun, testArray[i])
        bestScore = np.min([bestScore, np.sum(newFunArray), xorTree(bestScore, newFunArray, testArray[i+1:])])

    return bestScore

def xorTest(funArray: np.array) -> np.array:
    resultArray = np.array([])
    testArray = np.transpose(generateFunctions(SBoxLen, SBoxFunLen))

    if len(testArray) != SBoxLen:
        print("Error with shape!")
        exit(0)
    
    testArray = np.append(testArray, testArray^1, axis=0)

    for fun in funArray:
        resultArray = np.append(resultArray, [xorTree(np.inf, fun, testArray)], axis=0)
    
    return resultArray.astype("int32")

In order to test the sbox located into our file, we had to determine `Hamming Distance`. This is the minumum number of ones between each function and each posible output function. Algorythm which I prepared, is based on recursion. For each function, the xor function is examined, and it is checked whether the value of the ones is less than the currently smallest value. At each step, combinations based on the current modified function are also compared. Algorythm passes to next recursive steps only consecutive functions (order is established at the beginnig). This is because we want to avoid additional calculations of the same values. `None` value is representation of stopping state.

![](./img/sbox_alg.jpg)

In [6]:
print(xorTest(funArray))

[112 112 110 110 110 112 110 110]


For each function, we got a value 112 or 110 which was expected.

In [7]:
# def main():
#     funArray = createFunctions()
#     print(np.sum(funArray, axis=1))
#     print(xorTest(funArray))

# if __name__ == "__main__":
#     main()

### Souces
* https://www.geeksforgeeks.org/difference-between-aes-and-des-ciphers/
* https://en.wikipedia.org/wiki/S-box

